Jacob Mark Hunter  Project started 28/05/2025

Environmental monitoring using machine vision project

This project was inspired by the roboflow article names "how to use computer vision for environmental monitoring": https://blog.roboflow.com/computer-vision-environmental-impact-monitoring/

Lecture content provided by UWE and Dr Desi Chen was also used

imports tensor flow

In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import numpy as np
import skimage.util as util
from scipy import ndimage
from skimage.exposure import rescale_intensity
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
import pandas as pd

dataset used: Deforestation Non-Deforestation Area Analysis by Akilesh G A on 29/05/2025 from Kaggle


I then created a CSV file to sort all of the images so that they where labled deforestation or no deforestation and if theyre for training or testing

Image organisation based on Dr desi Chen's lecture slides

In [ ]:


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# 1) Define paths
dataset_path   = '/content/drive/MyDrive/datasets/deforestation dataset/'
csv_label_path = os.path.join(dataset_path, 'Labels.csv')

# 2) Read the CSV (no header), then name columns:
df = pd.read_csv(csv_label_path, header=None)
df.columns = ['Usage', 'Label', 'Image_name']

# 3) Split train/test
train_df = df[df['Usage'] == 'TRAIN'].reset_index(drop=True)
test_df  = df[df['Usage'] == 'TEST'].reset_index(drop=True)

mun_train = len(train_df)
mun_test  = len(test_df)

# 4) Pre‐allocate as 1-D label arrays (not (N,1))
train_images = np.zeros((mun_train, 128, 128, 3), dtype=np.float32)
train_labels = np.zeros((mun_train,),         dtype=np.int32)

test_images  = np.zeros((mun_test, 128, 128, 3), dtype=np.float32)
test_labels  = np.zeros((mun_test,),          dtype=np.int32)

# 5) Build train set
for i in range(mun_train):
    train_name = train_df.iloc[i, 2]                  # e.g. "deforestation/img_0001.png"
    train_path = os.path.join(dataset_path, train_name)

    img = plt.imread(train_path)  # can be (H,W), (H,W,3), or (H,W,4)

    # 5A) Drop alpha if present:
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]

    # 5B) If grayscale (2-D), turn into 3 channels:
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)

    # 5C) Normalize to [0,1] float32
    img = img.astype('float32')
    if img.max() > 1.0:
        img /= 255.0

    # 5D) Resize to 128×128:
    img = tf.image.resize(img, [128, 128]).numpy().astype('float32')

    train_images[i] = img
    train_labels[i] = int(train_df.iloc[i, 1])  # should be 0 or 1

# 6) Build test set (same steps, but no “if” around resize/assign)
for i in range(mun_test):
    test_name = test_df.iloc[i, 2]
    test_path = os.path.join(dataset_path, test_name)

    img = plt.imread(test_path)

    # 6A) Drop alpha channel if it exists:
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]

    # 6B) Grayscale → 3-channel
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)

    # 6C) Normalize
    img = img.astype('float32')
    if img.max() > 1.0:
        img /= 255.0

    # 6D) Resize to 128×128
    img = tf.image.resize(img, [128, 128]).numpy().astype('float32')

    test_images[i] = img
    test_labels[i] = int(test_df.iloc[i, 1])      # 0 or 1

# 7) Sanity‐checks
import numpy as _np
print("train_images.shape:", train_images.shape, " dtype:", train_images.dtype)
print("train_labels.shape:", train_labels.shape, " dtype:", train_labels.dtype)
print("unique train_labels:", _np.unique(train_labels), " min/max:", train_labels.min(), train_labels.max())

print("test_images.shape:", test_images.shape, " dtype:", test_images.dtype)
print("test_labels.shape:", test_labels.shape, " dtype:", test_labels.dtype)
print("unique test_labels:", _np.unique(test_labels), " min/max:", test_labels.min(), test_labels.max())


train_images.shape: (104, 128, 128, 3)  dtype: float32
train_labels.shape: (104,)  dtype: int32
unique train_labels: [0 1]  min/max: 0 1
test_images.shape: (25, 128, 128, 3)  dtype: float32
test_labels.shape: (25,)  dtype: int32
unique test_labels: [0 1]  min/max: 0 1


In [ ]:
# Suppose train_labels is currently shape (104, 1)
train_labels = train_labels.squeeze()   # now shape is (104,)
# —or—
train_labels = train_labels.reshape(-1) # also yields shape (104,)

In [ ]:
print("dtype of train_labels:", train_labels.dtype)
print("shape of train_labels:", train_labels.shape)
print("unique values in train_labels:", np.unique(train_labels))
print("min, max:", train_labels.min(), train_labels.max())

dtype of train_labels: int32
shape of train_labels: (104,)
unique values in train_labels: [0 1]
min, max: 0 1


Creation of CNN

In [ ]:
#conv2D and pooling layers
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#hidden dense layers
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 213,738 (834.91 KB)

 Trainable params: 213,738 (834.91 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#model training
model.compile(optimizer='adam',
       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
       metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 648ms/step - accuracy: 0.4792 - loss: 0.6783 - val_accuracy: 0.6000 - val_loss: 0.6713
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 467ms/step - accuracy: 0.6155 - loss: 0.6350 - val_accuracy: 0.6000 - val_loss: 0.6308
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 489ms/step - accuracy: 0.7673 - loss: 0.5921 - val_accuracy: 0.6000 - val_loss: 0.5857
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 499ms/step - accuracy: 0.6912 - loss: 0.5208 - val_accuracy: 0.6800 - val_loss: 0.5458
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 893ms/step - accuracy: 0.8817 - loss: 0.4391 - val_accuracy: 0.7200 - val_loss: 0.7072
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 598ms/step - accuracy: 0.8629 - loss: 0.3564 - val_accuracy: 0.8800 - val_loss: 0.4013
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 604ms/step - accuracy: 0.8030 - loss: 0.3839 - val_accuracy: 0.8400 - val_loss: 0.4622
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 558ms/step - accuracy: 0.8966 - loss: 0.2623 - val_accuracy: 0.7200 - val_loss: